In [1]:
from bs4 import BeautifulSoup
import urllib
import http.cookiejar
from datetime import datetime
from calendar import timegm




class HTTPRedirectHandler(urllib.request.HTTPRedirectHandler):
    def __init__(self, jar):
        self.jar = jar

    def redirect_request(self, req, fp, code, msg, hds, newurl):

        sessionid = ''
        for i in hds._headers:
            if i[0] != 'Set-Cookie':
                continue
            if i[1].startswith('sessionid='):
                sessionid = i[1]
                print("In redirect_request(): SessionId found value is: " + sessionid + "\n\n")
                break

        newreq = super().redirect_request(
            req, fp, code, msg, hds, newurl)


        return newreq



def get_url(url, data=None, timeout=30, opener=None, requestHeader = None, jar = None):
  '''get_url accepts a URL string and return the server response code, response headers, and contents of the file'''

  req_headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36'
  }

  if requestHeader:
      req_headers.update(requestHeader)

  request = urllib.request.Request(url, headers=req_headers, data=data)
  if not opener:
    jar = http.cookiejar.CookieJar()
    redirectHandler = HTTPRedirectHandler(jar)


    opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor(jar), redirectHandler)


  while 1:
    try:
        response = opener.open(request)
    except Exception as inst:
        print(type(inst))
        print(inst.args)
        print(inst)

    break

  jar.extract_cookies(response, request)
  print("In get_url(): ")
  print(jar.make_cookies(response, request))

  code = response.code
  headers = response.headers
  html = response.read()
  return code, headers, html, opener, jar



code, headers, html, cur_opener, jar = get_url('https://www.screener.in/login/', timeout=3)
# print(html)

#responceHeaders = headers['Set-Cookie']
#responceHeaders = responceHeaders[responceHeaders.find('=') + 1:responceHeaders.find(';')]
#print(responceHeaders)
#print(headers)

# soup = BeautifulSoup(html, 'html.parser')
# fdata = soup.find_all('input', {'name': "csrfmiddlewaretoken"})
# csrfMiddlewareToken = str(fdata[0])
# csrfMiddlewareToken = csrfMiddlewareToken[csrfMiddlewareToken.find('value="') + len('value="') : csrfMiddlewareToken.find('>') -1]
# print("The value of csrfMiddlewareToken is: " + csrfMiddlewareToken + "\n\n")
# requestHeaderL ={ 'referer' : 'https://www.screener.in/login/', 'origin' : 'https://www.screener.in', 'Connection' : "keep-alive", 'sec-fetch-mode': 'navigate', 'sec-fetch-site': 'same-origin', 'sec-fetch-user': '?1'}

login_data = urllib.parse.urlencode({ 'csrfmiddlewaretoken' : csrfMiddlewareToken, 'next': '', 'username' : "shaik.imran@gmail.com", 'password' : "iiit123", 'action' : 'login'}).encode('UTF-8') 
code, headers, html, cur_opener, jar = get_url('https://www.screener.in/login/', data=login_data, timeout=3, opener= cur_opener, requestHeader=requestHeaderL, jar=jar)
print("The value of respond headers is: ")
print(headers)

code, headers, html, cur_opener, jar = get_url('https://www.screener.in/company/BHARATRAS/',timeout=3, opener= cur_opener, requestHeader=requestHeaderL, jar= jar)
soup= BeautifulSoup(html,'html.parser')
fdata = soup.find_all('li')
print(fdata)

# for i in fdata:
#     strVal = i.get_text().replace('\n','').strip()
#     val = strVal[0:strVal.find(':')].strip()
#     if val != 'PEG Ratio':
#         continue
#     else:
#         print('\nPEG Ratio found\n')
#         break;
# else:
#     print('\nPEG Ratio not found')

In get_url(): 
[Cookie(version=0, name='csrftoken', value='OjJ0cuuSpknB6YH5UrGrwQ5HOkyxcrnK', port=None, port_specified=False, domain='www.screener.in', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=True, expires=1764868170, discard=False, comment=None, comment_url=None, rest={'SameSite': 'Lax'}, rfc2109=False)]


NameError: name 'csrfMiddlewareToken' is not defined

In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

SCREENER_TOP_DATA = ['Market Cap', 'Current Price', 'High / Low', 'Stock P/E', 'Book Value',
                     'Dividend Yield', 'ROCE', 'ROE', 'Face Value', 'ROIC', 'PEG Ratio',
                     'Debt to equity', 'Financial leverage', 'EVEBITDA', 'Pledged percentage',
                     'CMP / FCF']

def fetch_number_span(list_element):
    num_span = list_element.find('span',{'class':'number'})

    num_span = num_span.text.replace(',', '')
    return float(num_span) if (num_span != '') else 0.0
    
def extract_scrip_ratios(soup,div_class, ul_id):
    div_html = soup.find('div',{'class': div_class})
    ul_html = div_html.find('ul',{'id': ul_id})
     
    scrip_data = pd.Series()
    for d in SCREENER_TOP_DATA:
        print(d)
        for li in ul_html.find_all("li"):
            name_span = li.find('span',{'class':'name'})
            
            if d in name_span.text: 
                scrip_data[d] = fetch_number_span(li)

    return scrip_data

login_url = 'https://www.screener.in/login/'
data = {
    'id_username': 'shaik.imran@gmail.com',
    'id_password': 'iiit123'
}

with requests.Session() as s:
    response = s.post(login_url , data)
    print(response.status_code) # 403 - Forbidden
#     index_page = s.get('https://www.screener.in/company/BERGEPAINT/consolidated/')
#     soup = BeautifulSoup(index_page.text, 'html.parser')
#     scrip_data = extract_scrip_ratios(soup,'company-ratios', 'top-ratios')
#     print(scrip_data)

from urllib.request import Request, urlopen

req = Request(
    url='http://www.cmegroup.com/trading/products/#sortField=oi&sortAsc=false&venues=3&page=1&cleared=1&group=1', 
    headers={'User-Agent': 'Mozilla/5.0'}
)
webpage = urlopen(req).read()

403


KeyboardInterrupt: 

In [ ]:
type(webpage)

In [ ]:
div_class, ul_id = 'company-ratios', 'top-ratios'
div_html = soup.find('div',{'class': div_class})
ul_html = div_html.find('ul',{'id': ul_id})
ul_html.find_all("li")

In [ ]:
import requests

login_url = 'https://www.screener.in/login/'
payload = {'user_id': 'shaik.imran@gmail.com', 'password': 'iiit123'} 
session = requests.Session() # save an instance of request.Session 
req = session.post(login_url, data=payload, verify=False) 
 
data = session.get('https://www.screener.in/company/BERGEPAINT/consolidated/') # should not redirect you to the login page 

# print(dir(data))
soup = BeautifulSoup(data.text, 'html.parser')

# page=urlopen(req)
# soup = BeautifulSoup(page)
scrip_data = extract_scrip_ratios(soup,'company-ratios', 'top-ratios')
print(scrip_data)

In [ ]:
import requests
# from web_commons import extract_table_by_class, flatten_df, extract_compound_tables
# from web_commons import fetch_scrip_data, extract_scrip_ratios, get_screener_data
# from commons import read_xls, get_stock_data, NSE_BSE, save_csv, get_holding_quantities
# from consts import CONSOLIDATED_NOT_AVAILABLE_ON_SCREENER, OUT_DIR
# from urllib.request import urlopen, Request

SCREENER_TOP_DATA = ['Market Cap', 'Current Price', 'High / Low', 'Stock P/E', 'Book Value',
                     'Dividend Yield', 'ROCE', 'ROE', 'Face Value', 'ROIC', 'PEG Ratio',
                     'Debt to equity', 'Financial leverage', 'EVEBITDA', 'Pledged percentage',
                     'CMP / FCF']

def fetch_number_span(list_element):
    num_span = list_element.find('span',{'class':'number'})

    num_span = num_span.text.replace(',', '')
    return float(num_span) if (num_span != '') else 0.0
    
def extract_scrip_ratios(soup,div_class, ul_id):
    div_html = soup.find('div',{'class': div_class})
    ul_html = div_html.find('ul',{'id': ul_id})
     
    scrip_data = pd.Series()
    for d in SCREENER_TOP_DATA:
        print(d)
        for li in ul_html.find_all("li"):
            name_span = li.find('span',{'class':'name'})
            
            if d in name_span.text: 
                scrip_data[d] = fetch_number_span(li)

    return scrip_data


cookies = {
    '_ga': 'GA1.2.1753029431.1611727839',
    'csrftoken': 'ae5rUfn2PYjAxSIrAg32eZk37U7C2vyV',
    'sessionid': '9tgnw2y6ievud7362gb83gri2qopwab9',
}

headers = {
    'authority': 'www.screener.in',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'max-age=0',
    # 'cookie': '_ga=GA1.2.1753029431.1611727839; csrftoken=ae5rUfn2PYjAxSIrAg32eZk37U7C2vyV; sessionid=9tgnw2y6ievud7362gb83gri2qopwab9',
    'dnt': '1',
    'referer': 'https://www.screener.in/company/RAJRATAN/consolidated/',
    'sec-ch-ua': '"Chromium";v="116", "Not)A;Brand";v="24", "Google Chrome";v="116"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36',
}

response = requests.get('https://www.screener.in/company/BERGEPAINT/consolidated/', auth=('shaik.imran@gmail.com', 'iiit123'), cookies=cookies, headers=headers)
# req = Request('https://www.screener.in/company/BERGEPAINT/consolidated/', cookies=cookies, headers=headers)
# print(response.text)
soup = BeautifulSoup(response.text, 'html.parser')

# page=urlopen(req)
# soup = BeautifulSoup(page)
scrip_data = extract_scrip_ratios(soup,'company-ratios', 'top-ratios')
print(scrip_data)

In [ ]:
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
import pandas as pd
# from requests_html import HTMLSession

CONSOLIDATED_NOT_AVAILABLE_ON_SCREENER = ['PRINCEPIPE', 'RELAXO']


SCREENER_TOP_DATA = ['Market Cap', 'Current Price', 'High / Low', 'Stock P/E', 'Book Value',
                     'Dividend Yield', 'ROCE', 'ROE', 'Face Value', 'ROIC', 'PEG Ratio',
                     'Debt to equity', 'Financial leverage', 'EVEBITDA', 'Pledged percentage',
                     'CMP / FCF']

def fetch_number_span(list_element):
    num_span = list_element.find('span',{'class':'number'})

    num_span = num_span.text.replace(',', '')
    return float(num_span) if (num_span != '') else 0.0
    
def extract_scrip_ratios(soup,div_class, ul_id):
    div_html = soup.find('div',{'class': div_class})
    ul_html = div_html.find('ul',{'id': ul_id})
     
    scrip_data = pd.Series()
    for d in SCREENER_TOP_DATA:
        print(d)
        for li in ul_html.find_all("li"):
            name_span = li.find('span',{'class':'name'})
            
            if d in name_span.text: 
                scrip_data[d] = fetch_number_span(li)

    return scrip_data

# df['ROIC']
scrip = 'BERGEPAINT'
print(f'********** {scrip}')
URL = f'https://www.screener.in/company/{scrip}/'

login_url = 'https://www.screener.in/login/'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36',
}

params = {
    'csrfmiddlewaretoken': '',
    'next': '/api/company/6596449/quick_ratios/',
    'username': 'shaik.imran@gmail.com',
    'password': 'iiit123',
}

if scrip not in CONSOLIDATED_NOT_AVAILABLE_ON_SCREENER:
    URL += 'consolidated/'
    
# with requests.Session() as s:
#     s.headers.update(headers)
#     res = s.get(URL)
#     soup = BeautifulSoup(res.text,"lxml")
#     params['csrfmiddlewaretoken'] = soup.select_one("input[name='csrfmiddlewaretoken']")['value']
#     s.headers['Origin'] = 'https://www.screener.in'
#     s.headers['Referer'] = 'https://www.screener.in/login/'
#     resp = s.post(login_url,data=params)
#     print(s.cookies.items())
#     resp = s.post(URL,data=params)
#     print(resp)

with requests.Session() as s:
    login_url = 'https://www.screener.in/login/'
    USERNAME = "shaik.imran@gmail.com"
    PASSWORD = "iiit123"

    s.get(login_url)
    csrftoken = s.cookies['csrftoken']
    print(s.cookies.items())

    login_data = dict(csrftoken=csrftoken, next='', username=USERNAME, password=PASSWORD)
    s.post(login_url, data=login_data, headers={"Referer": "https://www.screener.in/"})

    r = s.get(URL)
#     r.html.render(timeout=10, sleep=10)
#     print(r.text)
    soup = BeautifulSoup(r.text,"lxml")
    print(extract_scrip_ratios(soup,'company-ratios', 'top-ratios'))


#     hdr = {'User-Agent':'Mozilla/5.0'}
# #     req = Request(URL, data=params)
#     page= s.post(req, data=params)
#     soup = BeautifulSoup(page)
#     data = {'Scrip': scrip} 
#     data.update(extract_scrip_ratios(soup,'company-ratios', 'top-ratios'))
#     print(data)

In [ ]:
import pandas as pd
import requests

df = pd.read_html(
    requests.get("https://www.screener.in/company/BERGEPAINT/consolidated/").text,
    flavor="bs4",
)

# df.to_csv("last_table.csv", index=False)
df